In [80]:
import pandas as pd
import numpy as np

In [82]:
customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
category = pd.read_csv("product_category_name_translation.csv")

In [129]:
# Convert Date Columns
# Orders
order_date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in order_date_cols:
    orders[col] = pd.to_datetime(orders[col])

In [131]:
# Reviews
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

In [133]:
# Order_items 
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])

In [135]:
# Filter Only Delivered Orders
orders = orders[orders['order_status'] == 'delivered']
# Filtering only delivered orders to ensure revenue and delivery metrics are accurate.

In [87]:
# Create Delivery Features
orders['delivery_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_purchase_timestamp']
).dt.days

orders['delivery_delay_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_estimated_delivery_date']
).dt.days

orders['delayed_flag'] = np.where(orders['delivery_delay_days'] > 0, 1, 0)

In [139]:
# Create Order-Level Revenue Table
# First create total item value
order_items['total_item_value'] = order_items['price'] + order_items['freight_value']

In [141]:
# Aggregate to Order Level
order_items_agg = order_items.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum',
    'total_item_value': 'sum',
    'order_item_id': 'count'
}).reset_index()

order_items_agg.rename(columns={
    'price': 'total_price',
    'freight_value': 'total_freight',
    'order_item_id': 'total_items'
}, inplace=True)

In [143]:
# Aggregate Payments (Order Level)
payments_agg = payments.groupby('order_id').agg({
    'payment_value': 'sum',
    'payment_installments': 'max'
}).reset_index()
# Why max installments?
#Because some orders have split payments.

In [145]:
# Merge Category Translation
products = products.merge(
    category,
    how='left',
    on='product_category_name'
)

In [147]:
# Rename
products.rename(columns={
    'product_category_name_english': 'product_category'
}, inplace=True)

In [149]:
# Merge All Tables

In [151]:
# 1️⃣ Merge Orders + Customers
master_df = orders.merge(customers, on='customer_id', how='left')

In [153]:
# 2️⃣ Merge Order Revenue
master_df = master_df.merge(order_items_agg, on='order_id', how='left')

In [155]:
# 3️⃣ Merge Payments
master_df = master_df.merge(payments_agg, on='order_id', how='left')

In [157]:
# 4️⃣ Merge Reviews
master_df = master_df.merge(
    reviews[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

In [159]:
# Create Time Features
master_df['order_year'] = master_df['order_purchase_timestamp'].dt.year
master_df['order_month'] = master_df['order_purchase_timestamp'].dt.month
master_df['order_year_month'] = master_df['order_purchase_timestamp'].dt.to_period('M')

In [161]:
# Final Cleaning
master_df.isnull().sum()

order_id                           0
customer_id                        0
order_status                       0
order_purchase_timestamp           0
order_approved_at                 14
order_delivered_carrier_date       2
order_delivered_customer_date      8
order_estimated_delivery_date      0
delivery_days                      8
delivery_delay_days                8
delayed_flag                       0
customer_unique_id                 0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
total_price                        0
total_freight                      0
total_item_value                   0
total_items                        0
payment_value                      1
payment_installments               1
review_score                     646
order_year                         0
order_month                        0
order_year_month                   0
dtype: int64

In [165]:
master_df['review_given'] = master_df['review_score'].notnull().astype(int)

In [120]:
master_df.shape

(97007, 26)

In [122]:
master_df.describe()

,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_days,delivery_delay_days,delayed_flag,customer_zip_code_prefix,total_price,total_freight,total_item_value,total_items,payment_value,payment_installments,review_score,order_year,order_month,review_given
count,97007,96993,97005,96999,97007,96999.000000,96999.000000,97007.000000,97007.000000,97007.000000,97007.000000,97007.000000,97007.000000,97006.000000,97006.000000,96361.000000,97007.000000,97007.000000,97007.000000
mean,2018-01-01 15:37:47.535074816,2018-01-02 03:01:25.057674240,2018-01-04 21:13:02.244719360,2018-01-14 04:58:15.062763264,2018-01-25 09:32:15.732473088,12.099537,-11.879803,0.067645,35201.946293,136.895653,22.780098,159.675751,1.142598,159.705107,2.931551,4.155717,2017.543507,6.030152,0.993341
min,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-10-04 00:00:00,0.000000,-147.000000,0.000000,1003.000000,0.850000,0.000000,9.590000,1.000000,9.590000,0.000000,1.000000,2016.000000,1.000000,0.000000
25%,2017-09-13 19:55:58.500000,2017-09-14 07:25:28,2017-09-15 21:49:52,2017-09-25 20:11:17.500000,2017-10-05 00:00:00,6.000000,-17.000000,0.000000,11355.000000,45.900000,13.850000,61.810000,1.000000,61.840000,1.000000,4.000000,2017.000000,3.000000,1.000000
50%,2018-01-20 13:20:35,2018-01-20 19:20:37,2018-01-24 01:14:44,2018-02-02 16:09:06,2018-02-16 00:00:00,10.000000,-12.000000,0.000000,24436.000000,86.000000,17.170000,105.220000,1.000000,105.240000,2.000000,5.000000,2018.000000,6.000000,1.000000
75%,2018-05-05 10:59:43,2018-05-05 17:10:23,2018-05-08 13:41:00,2018-05-15 20:03:15.500000,2018-05-28 00:00:00,15.000000,-7.000000,0.000000,59061.000000,149.900000,24.020000,176.200000,1.000000,176.200000,4.000000,5.000000,2018.000000,8.000000,1.000000
max,2018-08-29 15:00:37,2018-08-29 15:10:26,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-10-25 00:00:00,209.000000,188.000000,1.000000,99980.000000,13440.000000,1794.960000,13664.080000,21.000000,13664.080000,24.000000,5.000000,2018.000000,12.000000,1.000000
std,NaN,NaN,NaN,NaN,NaN,9.547594,10.182253,0.251136,29842.086519,208.698701,21.532577,218.442738,0.540014,218.461608,2.715884,1.284986,0.503642,3.229851,0.081333


In [124]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97007 entries, 0 to 97006
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       97007 non-null  object        
 1   customer_id                    97007 non-null  object        
 2   order_status                   97007 non-null  object        
 3   order_purchase_timestamp       97007 non-null  datetime64[ns]
 4   order_approved_at              96993 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97005 non-null  datetime64[ns]
 6   order_delivered_customer_date  96999 non-null  datetime64[ns]
 7   order_estimated_delivery_date  97007 non-null  datetime64[ns]
 8   delivery_days                  96999 non-null  float64       
 9   delivery_delay_days            96999 non-null  float64       
 10  delayed_flag                   97007 non-null  int32         
 11  customer_unique

In [163]:
#Save Clean Dataset 
master_df.to_csv("clean_master_dataset.csv", index=False)